In [5]:
%pip install -r ./requirements.txt

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement torch==1.13.1 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0)
ERROR: No matching distribution found for torch==1.13.1


In [1]:
from tonic.datasets.nmnist import NMNIST
from tonic.transforms import ToFrame

root_dir = "./NMNIST"
NMNIST(save_to=root_dir, train=True)
NMNIST(save_to=root_dir, train=False)

to_raster = ToFrame(sensor_size=(34, 34), n_time_bins=100)
dataset = NMNIST(save_to=root_dir, train=False, transform=to_raster)


ModuleNotFoundError: No module named 'tonic'

In [ ]:
from torch import nn

cnn = nn.Sequential(
    nn.Conv2d(2, 8, 3, padding=1, bias=False),
    nn.ReLU(),
    nn.AvgPool2d(2),
    nn.Conv2d(8, 16, 3, padding=1, bias=False),
    nn.ReLU(),
    nn.AvgPool2d(2),
    nn.Conv2d(16, 16, 3, padding=1, stride=2, bias=False),
    nn.ReLU(),
    nn.Flatten(),
    nn.Linear(256, 10, bias=False),
    nn.ReLU()
)


In [ ]:
from sinabs.from_torch import from_model

snn_convert = from_model(model=cnn, input_shape=(2, 34, 34), batch_size=4).spiking_model


In [ ]:
from sinabs.backend.dynapcnn import DynapcnnNetwork

cpu_snn = snn_convert.to("cpu")
dynapcnn = DynapcnnNetwork(snn=cpu_snn, input_shape=(2, 34, 34), discretize=True, dvs_input=False)
dynapcnn.to(device="speck2fmodule", chip_layers_ordering="auto")


In [ ]:
from sinabs.backend.dynapcnn.dynapcnn_visualizer import DynapcnnVisualizer

visualizer = DynapcnnVisualizer(window_scale=(4, 8), dvs_shape=(34, 34), spike_collection_interval=50)
visualizer.connect(dynapcnn)


In [ ]:
dvs_event = samna.speck2f.event.DvsEvent()
# 设置 dvs_event 的属性：x, y, t, p
# 然后发送到 devkit 推理
output = dynapcnn([dvs_event])
